In [1]:
import random
import torch
from sklearn.model_selection import KFold
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SCNNB(nn.Module):
    def __init__(self):
        super(SCNNB, self).__init__()

        self.conv1 = nn.Conv2d(1,32,kernel_size=(7,7))
        self.BN1 = nn.BatchNorm2d(32,False)
        self.pooling1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(32,64,kernel_size=(7,7))
        self.BN2 = nn.BatchNorm2d(64,False)
        self.pooling2 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(8640,1280)
        self.fc2 = nn.Linear(1280,1)
        
    def forward(self,x):
        
        x = self.pooling1(F.relu(self.conv1(x)))
        
        
        x = self.pooling2(F.relu(self.conv2(x)))
        
        x = x.reshape(-1,8640)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = self.fc2(x)  
        
        return x


In [3]:
#导入数据
#rest1
datapath1=r'D:\JQ_YJS\飞行试验数据\处理后\rest1.npy' 
data1=np.load(datapath1)
#print(data1.shape)
data1=torch.tensor(data1).unsqueeze(1)
print(data1.shape)
data1=data1.numpy()

#rest2
datapath2=r'D:\JQ_YJS\飞行试验数据\处理后\rest1.npy'
data2=np.load(datapath2)
#print(data2.shape)
data2=torch.tensor(data2).unsqueeze(1)
print(data2.shape)
data2=data2.numpy()
data_all=np.concatenate((data1,data2),axis=0)
#标签制作
label_all = torch.cat([torch.zeros(6344), torch.ones(6344)]).long()  
print(label_all.unique())
print(data_all.shape)


torch.Size([6344, 1, 30, 200])
torch.Size([6344, 1, 30, 200])
tensor([0, 1])
(12688, 1, 30, 200)


In [ ]:
# 创建十折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 保存结果的列表
historys = []
test_pred = []
test_real = []
accuracy, precision, recall, f1score = [], [], [], []
batchsz = 16
num_epochs = 100
# 进行十折交叉验证
for fold, (train_ind, test_ind) in enumerate(kfold.split(data_all, label_all)):
    print('fold号:', fold + 1)

    # 每一折验证前都要打乱训练集样本顺序
    n = len(train_ind)
    A = np.linspace(0, n - 1, n, dtype=int)
    random.shuffle(A)

    # 构建训练集、验证集、测试集
    epoch_train = data_all[train_ind[A[:int(0.8 * n)]]]
    epoch_val = data_all[train_ind[A[int(0.8 * n):]]]
    epoch_test = data_all[test_ind]
    label_train = label_all[train_ind[A[:int(0.8 * n)]]]
    label_val = label_all[train_ind[A[int(0.8 * n):]]]
    label_test = label_all[test_ind]

    # 转换为Tensor并创建DataLoader
    train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
    
    train_loader = DataLoader(train_dataset, batch_size=batchsz, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsz, shuffle=False)

    # 选择、创建模型
    model = SCNNB()
    #print(model)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # 配置模型训练
    criterion = nn.BCEWithLogitsLoss()  # 使用交叉熵损失
    
    optimizer = optim.SGD(model.parameters(), lr=1e-3)
    

    # 开始训练模型
    print('开始训练!!')
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(num_epochs):  # 训练100个epoch
        model.train()
        running_loss = 0.0

        # 训练阶段
        for inputs, targets in train_loader:
            #print(f"训练批次输入数据形状: {inputs.shape}")
            inputs = inputs.to(device)
            
            targets = targets.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            targets = targets.view(-1).float() 
            #targets=targets.view(-1,1).float()
            loss = criterion(outputs.view(-1), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        #print('Epoch:{}/{}'.format(epoch+1,num_epochs))
        
        # 计算训练集的指标
        with torch.no_grad():
            train_pred = model(torch.tensor(epoch_train, dtype=torch.float32).to(device))
            train_pred = (torch.sigmoid(train_pred) > 0.5).cpu().numpy().astype(int)
            
            acc_train = accuracy_score(label_train, train_pred)
            pre_train = precision_score(label_train, train_pred, average='macro')
            rec_train = recall_score(label_train, train_pred, average='macro')
            f1_train = f1_score(label_train, train_pred, average='macro')

        
       
        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                

                inputs = inputs.to(device)
            
                targets = targets.to(device)
                outputs = model(inputs)
                #targets=targets.view(-1,1).float()
                targets = targets.view(-1).float()
                loss = criterion(outputs.view(-1), targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        
        history['val_loss'].append(avg_val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, VAL Loss: {avg_val_loss:.4f}')
        #print(f'Epoch [{epoch+1}/{num_epochs}], VAL Loss: {avg_val_loss:.4f}')
        print(f"$$训练集准确率 accuracy: {acc_train}",f"$$训练集精确率 precision: {pre_train}",f"$$训练集召回率 recall: {rec_train}",f"$$训练集 F1 评分 f1_score: {f1_train}")
        


        # 保存训练记录
        historys.append(history)
        

        # 计算、保存测试结果
        model.eval()
        with torch.no_grad():
            pred_test = model(torch.tensor(epoch_test, dtype=torch.float32).to(device))
            
            pred_test = (torch.sigmoid(pred_test) > 0.5).cpu().numpy().astype(int)


        # 保存预测结果和真实结果
        test_pred.append(pred_test)
        test_real.append(label_test)
        print(pred_test)
        # 计算准确率，精确率，召回率，F1评分
        acc = accuracy_score(label_test, pred_test)
        pre = precision_score(label_test, pred_test, average='macro')
        rec = recall_score(label_test, pred_test, average='macro')
        f1 = f1_score(label_test, pred_test, average='macro')
        
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        f1score.append(f1)
        
        print(f"@@测试集准确率 accuracy: {acc}",f"@@测试集精确率 precision: {pre}",f"@@测试集召回率 recall: {rec}",f"@@测试集 F1 评分 f1_score: {f1}")
        

# 将每一折 history 中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train = []
loss_val = []
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['train_loss'])

fold号: 1


C:\Users\FJQ\AppData\Local\Temp\ipykernel_10512\659748826.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_10512\659748826.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.6944, VAL Loss: 0.6938
$$训练集准确率 accuracy: 0.4975369458128079 $$训练集精确率 precision: 0.4964553794829024 $$训练集召回率 recall: 0.4982838626043202 $$训练集 F1 评分 f1_score: 0.4236377889147948
[[0]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
@@测试集准确率 accuracy: 0.5106382978723404 @@测试集精确率 precision: 0.5242736926467642 @@测试集召回率 recall: 0.5120372394332611 @@测试集 F1 评分 f1_score: 0.44096400755644816
Epoch [2/100], Train Loss: 0.6932, VAL Loss: 0.6938
$$训练集准确率 accuracy: 0.5004926108374385 $$训练集精确率 precision: 0.5035949029039389 $$训练集召回率 recall: 0.5013199139525009 $$训练集 F1 评分 f1_score: 0.40719484443463716
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
@@测试集准确率 accuracy: 0.508274231678487 @@测试集精确率 precision: 0.5277482517482518 @@测试集召回率 recall: 0.5098563281203425 @@测试集 F1 评分 f1_score: 0.41487550542668655
Epoch [3/100], Train Loss: 0.6941, VAL Loss: 0.6946
$$训练集准确率 accuracy: 0.5047619047619047 $$训练集精确率 precision: 0.5050341751733826 $$训练集召回率 recall: 0.5049191432868791 $$训练集 F1 评分 f1_score: 0.5019942528